In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data4 = data[data['PdlCyclist'] == 1]
filtered_data4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12354 entries, 32 to 54035
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       12354 non-null  int64  
 1   BedDys           12354 non-null  int64  
 2   BeddystoCC       12354 non-null  float64
 3   Severity         12354 non-null  int64  
 4   DiedCases        12354 non-null  int64  
 5   DiedBdDys        12354 non-null  int64  
 6   Hsptlsns         12354 non-null  int64  
 7   April            12354 non-null  int64  
 8   August           12354 non-null  int64  
 9   December         12354 non-null  int64  
 10  February         12354 non-null  int64  
 11  January          12354 non-null  int64  
 12  July             12354 non-null  int64  
 13  June             12354 non-null  int64  
 14  March            12354 non-null  int64  
 15  May              12354 non-null  int64  
 16  November         12354 non-null  int64  
 17  October         

In [4]:
num_bins = 2

bed_days_min = filtered_data4['Severity'].min()
bed_days_max = filtered_data4['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_2289/3159902379.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)


In [5]:
from xgboost import XGBClassifier

X = filtered_data4[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data4['Bed_days_category']

XGBOOST 

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before calibration
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(xgb_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predictions and probabilities after calibration
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    confusion_matrices_after.append(confusion_matrix(y_test, y_pred_after))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Calibration:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_after)} After Calibration:\n', confusion_matrices_after[-1])

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix before and after calibration
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0).astype(int)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0).astype(int)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Confusion Matrix for Fold 1 Before Calibration:
 [[1223    4]
 [   1    8]]
Confusion Matrix for Fold 1 After Calibration:
 [[1222    5]
 [   0    9]]
Confusion Matrix for Fold 2 Before Calibration:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 2 After Calibration:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 3 Before Calibration:
 [[1224    2]
 [   3    7]]
Confusion Matrix for Fold 3 After Calibration:
 [[1220    6]
 [   1    9]]
Confusion Matrix for Fold 4 Before Calibration:
 [[1224    2]
 [   1    9]]
Confusion Matrix for Fold 4 After Calibration:
 [[1224    2]
 [   0   10]]
Confusion Matrix for Fold 5 Before Calibration:
 [[1223    3]
 [   3    6]]
Confusion Matrix for Fold 5 After Calibration:
 [[1222    4]
 [   3    6]]
Confusion Matrix for Fold 6 Before Calibration:
 [[1225    1]
 [   3    6]]
Confusion Matrix for Fold 6 After Calibration:
 [[1225    1]
 [   0    9]]
Confusion Matrix for Fold 7 Before Calibration:
 [[1224    2]
 [   2    7]]
Confusion Matrix f

LOGESTIC REGRESSION

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    lr_model = LogisticRegression( random_state=42)
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_lr_before = lr_model.predict(X_test)
    y_prob_lr_before = lr_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_lr_before))
    precision_list_before.append(precision_score(y_test, y_pred_lr_before))
    recall_list_before.append(recall_score(y_test, y_pred_lr_before))
    f1_list_before.append(f1_score(y_test, y_pred_lr_before))
    entropy_list_before.append(calculate_entropy(y_prob_lr_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_lr_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(lr_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_lr_after = calibrated_model.predict(X_test)
    y_prob_lr_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_lr_after))
    precision_list_after.append(precision_score(y_test, y_pred_lr_after))
    recall_list_after.append(recall_score(y_test, y_pred_lr_after))
    f1_list_after.append(f1_score(y_test, y_pred_lr_after))
    entropy_list_after.append(calculate_entropy(y_prob_lr_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_lr_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.996438557184597
Accuracy Std Dev: 0.0015857118127265695
Mean Precision: 0.7987518037518038
Precision Std Dev: 0.12801511402072627
Mean Recall: 0.72
Recall Std Dev: 0.1572761980375155
Mean F1-score: 0.7472937245073468
F1-score Std Dev: 0.11272685406439722
Mean Entropy: 0.015517722994143024
Entropy Std Dev: 0.0014223813505648366
Mean Confusion Matrix:
 [[1224.4    1.8]
 [   2.6    6.6]]
--- After Calibration ---
Mean Accuracy: 0.9961145395228176
Accuracy Std Dev: 0.002255243203286869
Mean Precision: 0.6868486170692053
Precision Std Dev: 0.12967131183376066
Mean Recall: 0.9455555555555556
Recall Std Dev: 0.07377811244904013
Mean F1-score: 0.7916024402752091
F1-score Std Dev: 0.10749834293010417
Mean Entropy: 0.010987187211639538
Entropy Std Dev: 0.0014752339666500652
Mean Confusion Matrix:
 [[1.2219e+03 4.3000e+00]
 [5.0000e-01 8.7000e+00]]


Decesion tree

In [8]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_dt_before = dt_model.predict(X_test)
    y_prob_dt_before = dt_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_dt_before))
    precision_list_before.append(precision_score(y_test, y_pred_dt_before))
    recall_list_before.append(recall_score(y_test, y_pred_dt_before))
    f1_list_before.append(f1_score(y_test, y_pred_dt_before))
    entropy_list_before.append(calculate_entropy(y_prob_dt_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_dt_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(dt_model, method='sigmoid', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_dt_after = calibrated_model.predict(X_test)
    y_prob_dt_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_dt_after))
    precision_list_after.append(precision_score(y_test, y_pred_dt_after))
    recall_list_after.append(recall_score(y_test, y_pred_dt_after))
    f1_list_after.append(f1_score(y_test, y_pred_dt_after))
    entropy_list_after.append(calculate_entropy(y_prob_dt_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_dt_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9944149207971386
Accuracy Std Dev: 0.0015117303721251668
Mean Precision: 0.6914285714285714
Precision Std Dev: 0.16577465375522402
Mean Recall: 0.4977777777777777
Recall Std Dev: 0.1541043417563183
Mean F1-score: 0.5609224598930482
F1-score Std Dev: 0.1367847708822106
Mean Entropy: 2.4025851012630826e-09
Entropy Std Dev: 3.203656736723526e-25
Mean Confusion Matrix:
 [[1223.9    2.3]
 [   4.6    4.6]]
--- After Calibration ---
Mean Accuracy: 0.9944958269460058
Accuracy Std Dev: 0.0007048606468474237
Mean Precision: 0.7248809523809524
Precision Std Dev: 0.127553787951835
Mean Recall: 0.4655555555555556
Recall Std Dev: 0.098438424696813
Mean F1-score: 0.5514543042097222
F1-score Std Dev: 0.08006029682272404
Mean Entropy: 0.02688749147265705
Entropy Std Dev: 0.0017744408826550668
Mean Confusion Matrix:
 [[1224.3    1.9]
 [   4.9    4.3]]


SVM

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base SVM model
    svm_model = SVC(probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = svm_model.predict(X_test)
    y_prob_before = svm_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(svm_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9962764173316039
Accuracy Std Dev: 0.0017059058608087957
Mean Precision: 0.7224908424908425
Precision Std Dev: 0.13052459135219677
Mean Recall: 0.8466666666666667
Recall Std Dev: 0.14140389573273807
Mean F1-score: 0.7717607565181263
F1-score Std Dev: 0.10603963231565842
Mean Entropy: 0.01076562166275168
Entropy Std Dev: 0.001236545088264615
Mean Confusion Matrix:
 [[1223.     3.2]
 [   1.4    7.8]]

After Calibration:
Mean Accuracy: 0.9963573889915228
Accuracy Std Dev: 0.0017829041616814804
Mean Precision: 0.7037518037518036
Precision Std Dev: 0.11112053238669527
Mean Recall: 0.9122222222222224
Recall Std Dev: 0.10869029509074664
Mean F1-score: 0.7908688453677012
F1-score Std Dev: 0.0994238960047102
Mean Entropy: 0.010289677987459517
Entropy Std Dev: 0.0015442438294423276
Mean Confusion Matrix:
 [[1.2225e+03 3.7000e+00]
 [8.0000e-01 8.4000e+00]]


Random Forest

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(rf_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9955480654586429
Accuracy Std Dev: 0.0009045599893081259
Mean Precision: 0.7826190476190475
Precision Std Dev: 0.10986515131707099
Mean Recall: 0.5766666666666667
Recall Std Dev: 0.07070194786636902
Mean F1-score: 0.6587531328320803
F1-score Std Dev: 0.062095116714121865
Mean Entropy: 0.010583208530163448
Entropy Std Dev: 0.0009450282742729195
Mean Confusion Matrix:
 [[1224.6    1.6]
 [   3.9    5.3]]

After Calibration:
Mean Accuracy: 0.9961954456716848
Accuracy Std Dev: 0.0017001840874223265
Mean Precision: 0.6995887445887445
Precision Std Dev: 0.11099921770786074
Mean Recall: 0.9011111111111111
Recall Std Dev: 0.09222222222222222
Mean F1-score: 0.7825985319028798
F1-score Std Dev: 0.08721032193651006
Mean Entropy: 0.008824479683956419
Entropy Std Dev: 0.0010433288810442617
Mean Confusion Matrix:
 [[1.2224e+03 3.8000e+00]
 [9.0000e-01 8.3000e+00]]
